# 001_Intersection_Over_Union

In [1]:
import torch

In [2]:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
  """
  Parameters: 
    boxes_preds (tensor): Predictions of bounding boxes (BATCH_SIZE x 4)
    boxes_labels (tensor): Ground truths of bounding boxes (BATCH_SIZE x 4)
    box_format (str): midpoint (x,y,w,h) or corners (x1,y1,x2,y2)

  Returns: 
    tensor: intersection over union over all examples
  """

  if box_format == "midpoint":
    box1_x1 = boxes_preds[:, 0:1] - boxes_preds[:, 2:3]/2
    box1_y1 = boxes_preds[:, 1:2] - boxes_preds[:, 3:4]/2
    box1_x2 = boxes_preds[:, 0:1] + boxes_preds[:, 2:3]/2
    box1_y2 = boxes_preds[:, 1:2] + boxes_preds[:, 3:4]/2

    box2_x1 = boxes_labels[:, 0:1] - boxes_labels[:, 2:3]/2
    box2_y1 = boxes_labels[:, 1:2] - boxes_labels[:, 3:4]/2
    box2_x2 = boxes_labels[:, 0:1] + boxes_labels[:, 2:3]/2
    box2_y2 = boxes_labels[:, 1:2] + boxes_labels[:, 3:4]/2

  if box_format == "corners":
    box1_x1 = boxes_preds[:, 0:1]
    box1_y1 = boxes_preds[:, 1:2]
    box1_x2 = boxes_preds[:, 2:3]
    box1_y2 = boxes_preds[:, 3:4]

    box2_x1 = boxes_labels[:, 0:1]
    box2_y1 = boxes_labels[:, 1:2]
    box2_x2 = boxes_labels[:, 2:3]
    box2_y2 = boxes_labels[:, 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

  intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

  box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
  box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

  return intersection / (box1_area + box2_area - intersection)

# 002_Non_Max_Suppression

### NMS Suppression Algo - 

1. Among all the bounding boxes we obtained remove all those bboxes 
  - where pc < probability_threshold.
    - pc = probability of confidence of object being present
2. while we have bboxes:
      - take out the largest pc box
      - remove all the other bboxes with IoU > threshold
3. Repeat step 2 for all classes

In [3]:
import torch

def non_max_suppression(bboxes, iou_threshold, prob_threshold, box_format="corners"):
  # bboxes: [[class,prob_confidence,x1,y1,x2,y2],[...],[...]]
  assert type(bboxes) == list

  bboxes = [box for box in bboxes if box[1] > prob_threshold]
  bboxes = sorted(bboxes, key=lambda x:x[1], reverse=True)
  bboxes_after_nms = []

  while bboxes:
    chosen_box = bboxes.pop(0)

    bboxes = [
              box 
              for box in bboxes 
              if box[0] != chosen_box[0] 
              or intersection_over_union(torch.tensor(chosen_box[2:]),
                                         torch.tensor(box[2:]),
                                         box_format=box_format) < iou_threshold
    ]

    bboxes_after_nms.append(chosen_box)

  return bboxes_after_nms

# 003_Mean_Average_Precision

In [6]:
import torch
from collections import Counter

In [11]:
def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5, box_format="corners", num_classes=20):
  """
    Parameters:
      pred_boxes (list): [[train_idx, class_pred, prob_score, x1, y1, x2, y2], ...]
  """
  average_precisions = []
  epsilon = 1e-6

  for c in range(num_classes):
    detections = []
    ground_truths = []

    for detection in pred_boxes:
      if detection[1] == c:
        detections.append(detection)
    
    for true_box in true_boxes:
      if true_box[1] == c:
        ground_truths.append(true_box)

    amount_bboxes = Counter([gt[0] for gt in ground_truths])

    for key, val in amount_bboxes.items():
      # amount_bboxes = {0: torch.tensor([0,0,0]), 1:torch.tensor([0,0,0,0,0])}
      amount_bboxes[key] = torch.zeros(val)
    
    detections.sort(key=lambda x:x[2], reverse=True)
    TP = torch.zeros(len(detections))
    FP = torch.zeros(len(detections))
    total_true_boxes = len(ground_truths)

    for detection_idx, detection in enumerate(detections):
      ground_truth_img = [bbox for bbox in ground_truths if bbox[0] == detection[0]]
      num_gts = len(ground_truth_img)
      best_iou = 0

      for idx, gt in enumerate(ground_truth_img):
        iou = intersection_over_union(torch.tensor(detection[3:]), 
                                      torch.tensor(gt[3:]), 
                                      box_format=box_format)
        if iou > best_iou:
          best_iou = iou
          best_gt_idx = idx
    
      if best_iou > iou_threshold:
        if amount_bboxes[detection[0]][best_gt_idx] == 0:
          TP[detection_idx] = 1
          amount_bboxes[detection[0]][best_gt_idx] = 1
        else:
          FP[detection_idx] = 1
      else:
        FP[detection_idx] = 1
    
    TP_cumsum = torch.cumsum(TP, dim=0)
    FP_cumsum = torch.cumsum(FP, dim=0)
    recalls = TP_cumsum / (total_true_boxes + epsilon)
    precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
    precisions = torch.cat((torch.tensor([1]), precisions))
    recalls = torch.cat((torch.tensor([0]), precisions))
    average_precisions.append(torch.trapz(precisions, recalls))

  return sum(average_precisions) /  len(average_precisions)

In [10]:
# Example of Counter
mp = Counter([0,0,0,1,1,1,1])
print(mp)   # image_1 has 4 bboxes, image_0 has 3 bboxes

Counter({1: 4, 0: 3})
